Prueba 2 31456812 ENRIQUE GONZALEZ

In [10]:

using Polynomials
using Plots
using CSV
using DataFrames
using Statistics
using Interpolations
using SymPy
using PyCall


function obtenerdatos(datos)
    promedios = []
    for fila in eachrow(datos)
        promedio = mean(fila[2:end])
        push!(promedios, promedio)
    end
    return promedios
end

#leo el csv
datos = CSV.read("datos.csv", DataFrame, header=0)

#agarro la primera columna y la temperaturas
dias = datos[:, 1] :

temperaturas = obtenerdatos(datos)

Base.Meta.ParseError: ParseError:
# Error @ c:\Users\guzen\OneDrive\Desktop\prueba 2 de calculo numerico\finalprueba2.ipynb:25:21
#agarro la primera columna y la temperaturas
#                   ┌
dias = datos[:, 1] :

┘ ── line break after `:` in range expression

INTERPOLACION DE TAYLOR


En este problema se suponen conocidos los valores de la funci´on f y sus
derivadas sucesivas hasta el orden n en el punto x0 y se trata de hallar un
polinomio Pn(x) de grado menor o igual que n tal que:

![Screenshot (57).png](<attachment:Screenshot (57).png>)

Bajo ciertas condiciones, este polinomio existe y se conoce como el
polinomio de Taylor de f de grado menor o igual que n en x0.

In [11]:
#busco primero la funcion Interpolada
funcion = LinearInterpolation(dias, temperaturas)
x = symbols("x")
#hice mi variable simbolica 

funcionaderivar = 0
for dia in dias
    terminos = funcion[dia]
    for numeroi in dias     
        if numeroi != dia
            numerador = (x - numeroi)
            denominador = (dia- numeroi)  
            terminos  *= numerador / denominador #consigo los terminos a acumular en mi funcion 
        end
    end
    funcionaderivar += terminos 
end

#declaro datos que necesito para comenzar el polinomio 
punto = 7; grado_taylor = 7;   

polinomio_taylor = 0
for k in 0:8
    derivada_k = diff(funcionaderivar, x, k)
    derivada_punto = subs(derivada_k, x, punto)
    termino_k = (derivada_punto/factorial(k)) * (x - punto)^k
    polinomio_taylor += termino_k
end

polimiotaylorfx = lambdify(polinomio_taylor, [x]) #me hace la funcion en base a x

x = range(minimum(dias), maximum(dias), length=50) ; y = polimiotaylorfx.(x) #saco los valores 

#grafico
scatter(dias, temperaturas, label="Datos Originales")
plot!(dias, temperaturas, label="Función Interpolada")
plot!(x, y, label="Polinomio de Taylor")


UndefVarError: UndefVarError: `dias` not defined

INTERPOLACION DE LAGRANGE 

La interpolación de Lagrange es un método numérico utilizado para aproximar funciones mediante un polinomio que pasa por un conjunto de puntos conocidos. Este polinomio es una combinación lineal de polinomios base de Lagrange, cada uno asociado a un punto conocido.

![Screenshot (58).png](<attachment:Screenshot (58).png>)

In [12]:
function lagrange_polynomial(xi, fi)
    n = length(xi)
    x = Polynomial([0, 1]) 
    polinomio = Polynomial([0])
    
    for i in 1:n
        numerador = Polynomial([1])
        denominador = 1
        for j in 1:n
            if j != i
                numerador *= x - xi[j]
                denominador *= xi[i] - xi[j]
            end
        end
        terminoLi = numerador / denominador
        polinomio += terminoLi * fi[i]
    end
    
    return polinomio
end

#saco el polinomio para despues evaluarlo

p_lagrange = lagrange_polynomial(dias, temperaturas)

x = range(minimum(dias), maximum(dias), length=50)
y = [p_lagrange(v) for v in x]

scatter(dias, temperaturas, label="Datos originales", color=:yellow)
plot!(dias, temperaturas, label = " union de los Puntos")
plot!(x, y, label="Interpolación de Lagrange", color=:purple)





UndefVarError: UndefVarError: `dias` not defined

INTERPOLACIÓN DE HERMITE 

La interpolación de Hermite es un método de análisis numérico que se utiliza para construir un polinomio que no solo pasa por un conjunto de puntos dados, sino que también coincide con las derivadas conocidas de una función en esos puntos. A diferencia de la interpolación de Lagrange, que solo considera los valores de la función, la interpolación de Hermite también toma en cuenta las derivadas, lo que resulta en un ajuste más preciso y suave de la curva interpolada.

![Screenshot (59).png](<attachment:Screenshot (59).png>)




In [13]:
using Plots
using Interpolations

function calcular_derivadas(x, y)
    n = length(x)
    dy = similar(y) #me da un arreglo del tamaaño de y para tomar las derivadas en cada punto
    for i in 1:n-1
        dy[i] = (y[i+1] - y[i]) / (x[i+1] - x[i]) #aqui se hace la "diferencia de derivadas centradas finitas"
    end
    dy[n] = dy[n-1]
    return dy
end

df = calcular_derivadas(dias, temperaturas)

function interpolaciondehermite(x, y, dy, xnuevo)
    iterados = searchsortedfirst(x, xnuevo) #me busca cuando coincidan 
    iterados = max(1, min(iterados, length(x)-1))
    h = x[iterados + 1] - x[iterados]
    t = (xnuevo - x[iterados]) / h
    t2 = t^2
    t3 = t^3

    a0 = y[iterados]
    a1 = dy[iterados]
    a2 = 3 * (y[iterados+1] - y[iterados]) / h^2 - 2 * dy[iterados] / h - dy[iterados+1] / h
    a3 = 2 * (y[iterados] - y[iterados+1]) / h^3 + (dy[iterados] + dy[iterados+1]) / h^2

    return a0 + a1 * t + a2 * t2 + a3 * t3
end

x = range(minimum(dias), maximum(dias), length=8)
y = [interpolaciondehermite(dias, temperaturas, df, x) for x in x]


diaprediccion = 8
predictempertura = [interpolaciondehermite(dias, temperaturas, df, diaprediccion)]
println("la prediccion para ese dia sobre la temperatura de ese dia: ",predictempertura," grados celsius")


scatter(dias,temperaturas, label = "temperaturas")
plot!(dias, temperaturas, label="union de los puntos")
plot!(x, y, label="interpolacion de hermite")


UndefVarError: UndefVarError: `dias` not defined

INTERPOLACION POLINOMICA A TROZOS

es una tecnica de analisis numerico para aproximar una funcion mediante polinomios de menor grado en subintervalos de los datos
![Screenshot (60).png](<attachment:Screenshot (60).png>)

In [14]:
function interpolacionPolinomialatrozos(dias, temperaturas, x)
    iterados = searchsortedfirst(dias, x)
    if iterados == 1
        return temperaturas[1]
    elseif iterados > length(dias)
        return temperaturas[end]
    else
        x1, x2 = dias[iterados-1], dias[iterados]
        y1, y2 = temperaturas[iterados-1], temperaturas[iterados]
        return y1 + (x - x1) * (y2 - y1) / (x2 - x1)
    end
end



interdias = range(dias[1], stop=dias[6], length=8)
tempinters = [interpolacion_lineal_trozos(dias, temperaturas, x) for x in interdias]

scatter(dias, temperaturas)
plot!(interdias, tempinters, label = " interpolacion polinomica", color =:black)
plot!(dias, temperaturas, color =:blue, label = "union")

UndefVarError: UndefVarError: `dias` not defined